# PANDAS TUTORIAL

In [2]:
import pandas as pd
import sys

In [3]:
#For parquet file format support
!pip install pyarrow
!pip install fastparquet

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
print(sys.version)
print('pandas version ', pd.__version__)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
pandas version  2.2.0


## I/O READING

In [5]:
#Usign pd to read files
# Extremely heavy
# I've remove data source because it's extremely heavy, so jut take this a reference for quick reading
df = pd.read_parquet("<your parquet file name>")

# DATAFRAME BASICS

In [ ]:
pd.set_option('display.max_columns', 200)

In [ ]:
df.head(20)

In [ ]:
df.tail()

In [ ]:
df.sample(5)
# df.sample(frac=0.2)


NameError: name 'df' is not defined

In [ ]:
# For assign state and reproduce in future usage
df.sample(frac=0.1, random_state=529)

In [ ]:
df.columns

In [ ]:
df.index

In [ ]:
df.info()
# Quick summary df.info(verbose = False)

In [ ]:
df.describe()
# Handle non numeric column
df[['Airline']].desribe()

In [ ]:
df.shape
len(df)

# Subsetting a Dataframe

df[['FlightDate','Airline','Origin']]

In [ ]:
# Because df.columns returna rray
df.columns[:5]

In [ ]:
#List comprehension
[c for c in df.columns if 'Times' in c]

In [ ]:
# Select subset of the dataframe which have dtype of x
df.select_dtypes('int')

# Filter Rows
df.log[] and df.iloc[]

In [ ]:
df.iloc[1,3]

In [ ]:
df.iloc[:5,:5]

In [ ]:
# Lay entire row
df.iloc[5]
df.iloc[[5]]

In [ ]:
# Select entire colum
df.iloc[:,1]
df.iloc[:, [1,2]]

In [ ]:
df.loc[:, ['Airline', 'Origin']]

In [ ]:
# Get series of data 
# This will compare each cells
df['Airline'] == 'Spirit Air Lines'

In [ ]:
df.loc[df['Airline'] == 'Spirit Air Lines']

In [ ]:
# Filer voi and va or statement
df.loc[(df['Airline'] == 'Spirit Air Lines') & (df['FlightDate']=='2021-09-09')]

In [ ]:
# Extremely useful when you want to get the 'reverse' result of that booleans series
~(df['Airline'] == 'Spirit Air Lines') & (df['FlightDate']=='2021-09-09')

In [ ]:
df.loc[~(df['Airline'] == 'Spirit Air Lines') & (df['FlightDate']=='2021-09-09')]

# FILTER USING QUERY

For more versatile use of filtering, we can bring the knowledge from query language to this method

In [ ]:
df.query('Airline == "Spirit Air Lines"')
df.query('(Deptime > 1130) and (Origin == "DRO")')

In [ ]:
min_time = 1130
df.query('(Deptime > @min_time) and (Origin == "Dro)')

# SUMMARY DATA

By understand basic stastic informtion fo the dataset we can deliever a more proper processing approach to the data
- Min
- Max
- Sum
- Mean
- Variant
- Standard deviation
- Quantile

These function will return Series and the values are statistic values with index(label) are the column name

In [ ]:
df['DepTime'].mean()
df['DepTime'].quantile([0.25, 0.75])
df['DepTime'].sum()

In [ ]:
type(df[['DepTime', 'DepDelay', 'ArrTime']].min())
# -> Series -> Can use numpy


In [ ]:
df[['DepTime', 'DepDelay', 'ArrTime']].agg(['mean','max','min'])
#  Send back datafram 

In [ ]:
# Ta cung co the su dung agg voi ca dictionary de chi cu the tung nhom du lieu minh muon aggrage the nao
df[['DepTime', 'DepDelay', 'ArrTime']].agg({
    'DepTime': ['min', 'max'],
    'DepDlay': ['mean'],
    'ArrTime': ['min', 'max']
})

* Summary on Categorical Variables

In [ ]:
df['Airlines'].unique()
df['Airlines'].value_counts()
df['Airlines'].nunique()
# Get the unique combination count - Multti index series
df[['Airline', 'Origin']].value_counts()

In [ ]:
df[['Airline', 'Origin']].value_counts().reset_index()


# ADVANCED COLUMN METHODS

In [ ]:
df.groupby('Airline')[['DepDelay']].mean()
# Grop by Airline and the groupby value will be the mean  of DepDleay

In [ ]:
df.groupby('Airline')[['DepDelay']].agg(['mean','max','mean'])

In [ ]:
df_agg=df.groupby('Airline')[['DepDelay', 'ArrDelay']].agg(['mean','max','mean'])
# This will return multi-index COLUMN*

In [ ]:
df_agg.columns
df_agg.columns.to_flat_index

In [ ]:
# Convert tat cac column ve flat column
df_agg.columns = ['_'.join(c) for c in df_agg.columns]

# NEW COLUMN

In [ ]:
df['DepTime2'] = df['DepTime'] / 60
df = df.assign(DepTime3 = df['DepTime'] / 60)

# SORTING

In [ ]:
df.sort_values('ArrDelay') 

In [ ]:
df[['FlightDate', 'Airline', 'ArrDelay']].sort_values('FlightDate', ascending=False).reset_index(drop=True)

# HANDLING MISSING DATA

In [ ]:
df[['FlightDate', 'Airline','ArrDelay']].isna().sum()

In [ ]:
df[['FlightDate', 'Airline','ArrDelay']].dropna()

In [ ]:
df[['FlightDate', 'Airline','ArrDelay']].dropna(subset=['ArrDelay'])

In [ ]:
df[['FlightDate', 'Airline','ArrDelay']].fillna(-999)

In [ ]:
df['Airline'].fillna(df['Airline'].mean())

# Combining data

In [ ]:
# Have to copy because Python using object - referenced
df1 = df.query('Airline == "Southwest Airlines Co."').copy()
df2 = df.query('Airline == "Delta Air Lines Inc."').copy()



In [ ]:
pd.concat([df1, df2])

In [ ]:
df_stack = pd.concat([df1, df2])

In [ ]:
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)

In [ ]:
df_side = pd.concat([df1, df2], axis=1)

In [ ]:

df1.shape, df2.shape, df_stack.shape, df_side.shape

# Merge dataframe

It will mere on Similar column (same name) on both dataframe

In [ ]:

df1 = df.groupby(['Airline', 'FlightDate'])[['DepDelay']].mean().reset_index()
df2 = df.groupby(['Airline', 'FlightDate'])[['ArrDelay']].mean().reset_index()


In [ ]:
df1.merge(df2, how="left")

In [ ]:
df1.merge(df2, how="right")

In [ ]:

df1.merge(df2, how="outer")

# Merge suffixes

In [ ]:
pd.merge(df1, df2, on=['Airline', 'FlightDate'])

In [ ]:

pd.merge(df1, df2, on=['Airline'])

In [ ]:
pd.merge(df1, df2, on=['Airline'], suffixes=('_dep', '_arr'))

In [ ]:
pd.merge(df1, df2, left_on=['Airline'], right_on=['Airline'])